# Interpolation using the RBF method

In [ ]:
# Domain is [-2,2] X [-2,2] = [a,b] X [c,d]

a=-2;
b=2;
c=-2;
d=2;

Nx=100; # Number of nodes on the x axis 
Ny=100; # Number of nodes on the y axis 

delta_x=(b-a)/(Nx-1); # Mesh size
delta_y=(d-c)/(Ny-1);

# Line space
x1=a:delta_x:b; 
y1=c:delta_y:d;


using LinearAlgebra

xd=repeat(transpose(x1),Nx,1);
yd=repeat(y1,1,Ny);

x=xd[:];
y=yd[:];

## Define the radial basis functions

In [ ]:
function phi(X1,X2,Y1,Y2,epsil)
    return exp.((-(epsil^2)*((X1-X2).^2+(Y1-Y2).^2)))
end

# Derivative with respective to x of phi
function phi_x(X1,X2,Y1,Y2,epsil)
    return 2*epsil^2*(X1-X2).*exp.(-(epsil^2)*((X1-X2).^2+(Y1-Y2).^2))
end

# Derivative with respective to y of phi
function phi_y(X1,X2,Y1,Y2,epsil)
    return 2*epsil^2*(Y1-Y2).*exp.(-(epsil^2)*((X1-X2).^2+(Y1-Y2).^2))
end

# Laplacian of phi
function phi_lap(X1,X2,Y1,Y2,epsil)
    return (-4*epsil^2*exp.(-(epsil^2)*((X1-X2).^2+(Y1-Y2).^2)))+(4*epsil^2*((X1-X2).^2+(Y1-Y2).^2).*exp.(-(epsil^2)*((X1-X2).^2+(Y1-Y2).^2)))
end



## Some Parameters

In [ ]:
N=size(x)[1];# Number of total nodes
epsil=2; # Shape parameter - epsilon

# Function that we interpolate
function f(x,y)
    return x.*exp.(-x.^2-y.^2)
end

## Evaluation Points

In [ ]:
N1=trunc(Int,Nx/2); # Number of nodes on the x axis 
N2=trunc(Int,Ny/2); # Number of nodes on the y axis 

delta_x1=(b-a)/(N1-1); # Mesh size
delta_y1=(d-c)/(N2-1);

# Line space
x2=a:delta_x1:b; 
y2=c:delta_y1:d;


using LinearAlgebra

x3=repeat(transpose(x2),N1,1);
y3=repeat(y2,1,N2);

xhat=x3[:];
yhat=y3[:];

Ne=size(xhat)[1];# Number of total evaluation nodes

In [ ]:
# meshgrid for RBF centers
XT1=repeat(transpose(x),N,1);
XT2=repeat(x,1,N);

YT1=repeat(transpose(y),N,1);
YT2=repeat(y,1,N);

# Collocation Matrix using the cartesian centers

A=phi(XT1,XT2,YT1,YT2,epsil);

# Second meshgrid for evaluation points

XTe1=repeat(transpose(x),Ne,1);
XTe2=repeat(xhat,1,N);

YTe1=repeat(transpose(y),Ne,1);
YTe2=repeat(yhat,1,N);

# Second collocation Matrix using the random evaluation points

Ahat=phi(XTe1,XTe2,YTe1,YTe2,epsil);
# Right-hand side
F=f(x,y);


## Find the interpolation coefficients

In [ ]:
L=A\F;
fhat=Ahat*L;

## Plot the solutions

In [ ]:
using Plots
plot(xhat,yhat,fhat,st=:surface)